# Classification Beer Recipes

In [2]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from io import StringIO

The dataset was extracted from [Kaggle](https://www.kaggle.com/datasets/jtrofe/beer-recipes), thanks to the [Brewer's Friend site](https://www.brewersfriend.com), which allows users to share their homebrew recipes. This dataset contains a selection of the recipes uploaded so far.

In [3]:
df_raw = pd.read_csv("../data/raw/recipeData.csv", encoding='latin1')
df_raw.sample(10, random_state=42)

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId
47060,47061,ISA prise 1,/homebrew/recipe/view/353541/isa-prise-1,American Pale Ale,10,18.93,1.039,1.008,4.07,31.01,...,1.055,35.0,NaN,Specific Gravity,extract,0.50,22.22,NaN,NaN,61376.0
26901,26902,Citra IPA,/homebrew/recipe/view/301441/citra-ipa,American IPA,7,18.93,1.053,1.009,5.71,44.28,...,1.040,70.0,NaN,Specific Gravity,BIAB,NaN,NaN,NaN,NaN,NaN
1165,1166,Clone - Two Hearted,/homebrew/recipe/view/5109/clone-two-hearted,American IPA,7,22.71,1.064,1.010,7.00,60.95,...,1.048,75.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,455.0
70995,70996,Double day pale,/homebrew/recipe/view/597569/double-day-pale,American Pale Ale,10,24.00,1.055,1.010,5.86,20.84,...,1.046,80.0,3.00,Specific Gravity,All Grain,NaN,18.00,NaN,NaN,43854.0
6656,6657,Amongst the Waves Summer Ale,/homebrew/recipe/view/199244/amongst-the-waves...,American Pale Ale,10,11.36,1.060,1.011,6.39,34.97,...,1.090,35.0,NaN,Specific Gravity,extract,0.35,18.33,NaN,NaN,40478.0
22928,22929,Dankey Kang IPA,/homebrew/recipe/view/239348/dankey-kang-ipa,American IPA,7,18.93,1.071,1.017,7.17,120.94,...,1.143,35.0,NaN,Specific Gravity,extract,NaN,20.00,NaN,NaN,NaN
48176,48177,Stout (Experiment),/homebrew/recipe/view/369918/stout-experiment-,Irish Stout,93,20.82,1.044,1.011,4.41,32.74,...,1.035,75.0,NaN,Specific Gravity,BIAB,0.75,18.89,NaN,NaN,43553.0
12639,12640,Farmhouse Ale New,/homebrew/recipe/view/426247/farmhouse-ale-new,Saison,134,20.82,1.044,1.012,4.30,26.44,...,1.035,74.0,1.75,Specific Gravity,All Grain,0.75,NaN,NaN,NaN,NaN
43582,43583,Another Simple Pilsner,/homebrew/recipe/view/435707/another-simple-pi...,Bohemian Pilsener,31,18.00,1.059,1.013,6.15,42.83,...,1.035,55.0,3.00,Specific Gravity,All Grain,1.50,11.00,NaN,NaN,49368.0
33657,33658,Vasily Modified,/homebrew/recipe/view/319594/vasily-modified,Imperial Stout,87,24.50,1.081,1.015,8.59,84.22,...,1.132,30.0,NaN,Specific Gravity,extract,0.35,20.00,corn sugar,200g,NaN


In [4]:
df_raw.shape

(73861, 23)

In [7]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73861 entries, 0 to 73860
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   BeerID         73861 non-null  int64  
 1   Name           73859 non-null  object 
 2   URL            73861 non-null  object 
 3   Style          73265 non-null  object 
 4   StyleID        73861 non-null  int64  
 5   Size(L)        73861 non-null  float64
 6   OG             73861 non-null  float64
 7   FG             73861 non-null  float64
 8   ABV            73861 non-null  float64
 9   IBU            73861 non-null  float64
 10  Color          73861 non-null  float64
 11  BoilSize       73861 non-null  float64
 12  BoilTime       73861 non-null  int64  
 13  BoilGravity    70871 non-null  float64
 14  Efficiency     73861 non-null  float64
 15  MashThickness  43997 non-null  float64
 16  SugarScale     73861 non-null  object 
 17  BrewMethod     73861 non-null  object 
 18  PitchR

### **Dictionary**
|name|description|units|type|
|----|-----------|-----|----|
|Beer ID|Record ID|NaN|int64|
|Name|Name of beer|NaN|object|
|URL|Location of recipe webpage at https://www.brewersfriend.com|NaN|object|
|Style|Type of brew|NaN|object|
|Style ID|Numeric ID for type of brew|NaN|int64|
|Size L|Amount brewed for recipe listed|Liters|float64|
|OG|The original gravity (sugar content) of the beer post Wort cooldown before pitching the yeast|SG|float64|
|FG|The final gravity (remaining sugar content) of the beer after fermentation is complete|SG|float64|
|ABV|Calculated alcohol by volume, which is determined from the difference between the OG and the FG|%|float64|
|IBU|International Bittering Units, which is how perceptively bitter the beer is (usually based on the hop addiitons and at what given time in the boil they were added)|IBU|float64|
|Color|Standard Reference Method - light to dark ex. 40 = black SRM (Standard Reference Method)|SRM|float64|
|Boil Size|Fluid at beginning of boil|Liters|float64|
|Boil Time|How long the wort was boiled|min|int64|
|Boil Gravity|Specific gravity of wort before the boil|SG|float64|
|Efficiency|How efficient the brew session was, which basically means how much possible sugars were extracted from the grains for fermenting|%|float64|
|MashThickness|Amount of water per pound of grain|L/kg|float64|
|SugarScale|Scale to determine the concentration of dissolved solids in wort|Brix|object|
|BrewMethod|Various techniques for brewing|NaN|object|
|PitchRate|Yeast added to the fermentor per gravity unit - M cells/ml/deg P|Millones de celulas/ml/P|float64|
|PrimaryTemp|Temperature at the fermenting stage|°C|float64|
|PrimingMethod|A method of carbonating beer, such as adding sugar or forced carbonation to create effervescence in the beverage|NaN|object|
|PrimingAmount|Amount of priming sugar used|g|object|
|User ID|Unique identifier of the user who created or shared the recipe.|NaN|float64|

## **Key variables**
|Variable|Description|Importance|Influence|
|--------|-----------|----------|---------|
|OG (Original Gravity)|Measures the sugar concentration in the wort before fermentation. A higher OG means a higher potential alcohol content and a fuller body.|	Determines the potential alcohol content and body of the beer, affecting its strength and texture.|Changes in OG can drastically alter the profile of the beer, as a heavier wort will result in a fuller-bodied beer with potentially more alcohol.|
|FG (Final Gravity)|Measures the remaining sugar concentration after fermentation is complete. The difference between OG and FG calculates ABV.|Impacts alcohol content and sweetness, contributing to mouthfeel and balance in the final beer.|FG affects mouthfeel and sweetness, important attributes in the final flavor of beer.|
|ABV (Alcohol by Volume)|Represents the percentage of alcohol, calculated from OG and FG. Higher ABV means a stronger beer.|Determines the strength and body of the beer, affecting the classification and sensory profile.|It affects both the perception of warmth on the palate and the level of body. It also influences the classification of beer styles.|
|IBU (International Bittering Units)|Indicates the bitterness level, determined by hop additions and boil time.|Affects the beer’s bitterness and balance with malt sweetness, essential in styles like IPAs.|A higher IBU can make the beer perceived as more bitter. It is important to find a suitable balance for the style of beer being brewed.|
|Style|Describes the beer type (e.g., Lager, IPA, Stout), which sets the expected taste, color, ABV, and IBU ranges.|Guides the expected characteristics, ensuring all other variables align with the desired beer type.|Ayuda a orientar las demás variables (como OG, FG, ABV, y IBU) para que se alineen con las expectativas del estilo.|
|Color (SRM)|Measures the beer’s color from light to dark on the Standard Reference Method (SRM) scale.|Important for meeting visual expectations of each style; dark beers often imply roasted or caramelized notes.|Color is key to matching the expectations of each style and contributes to the sensory experience of the beer.|
|Efficiency|Reflects how efficiently sugars are extracted from the grains during mashing.|Impacts the potential ABV and body, as higher efficiency yields more fermentable sugars.|It affects the final ABV, as increased efficiency can produce a beer with more alcohol or a more robust body, if the sugars are not fully fermented.|
|Primary Temp|Temperature during primary fermentation, influencing yeast activity and flavor profile.|Key for controlling the balance of desired flavors, such as esters and phenols, according to beer style.|Fermentation temperature is crucial to achieving the desired flavor profile and avoiding off-flavors.|

**Summary**
1. **OG and FG** – Determine alcohol content and body of the beer.
2. **ABV** – Important for strength classification and sensory perception.
3. **IBU** – Affects bitterness and overall flavor profile.
4. **Style** – Sets guidelines for expected beer characteristics.
5. **Color** – Enhances visual appeal and aligns with flavor expectations.
6. **Efficiency** – Influences ABV and body, affecting final taste.
7. **Primary Temp** – Controls flavor development during fermentation.

# Secundary keys variables
|Variable|Description|Importance|Influence|
|--------|-----------|----------|---------|
|Boil Size|Volume of liquid at the beginning of the boil.|Influences the concentration of flavors, bitterness (IBU), and affects the final wort gravity.|A larger boil size can dilute flavors and bitterness, while a smaller boil size increases concentration.|
|Boil Time|Duration of the boiling process.|Impacts bitterness extraction from hops, as longer boil times increase bitterness and flavor concentration.|Extended boil times lead to higher bitterness and color changes, enhancing hop flavors in hoppy styles like IPAs.|
|Boil Gravity|Specific gravity of the wort before boiling.|Helps in adjusting hops and other ingredient amounts for achieving the desired flavor and bitterness.|A higher boil gravity can result in stronger malt flavors and a thicker body, particularly important in strong ales.|
|Mash Thickness|Amount of water per pound of grain in the mash.|Affects the extraction of fermentable sugars, which in turn impacts OG, body, and mouthfeel.|Thicker mashes (less water) enhance body and sweetness, while thinner mashes increase fermentable sugars.|
|Sugar Scale|Scale used to measure dissolved solids in the wort, often Brix or Plato.|Standardizes measurement for gravity readings, helping with consistent adjustments in OG and FG.|Ensures precise gravity readings, which are essential for achieving consistent alcohol levels and body.|
|Brew Method|Brewing method (e.g., All Grain, Extract), which affects the complexity of flavors and efficiency.|Provides context for other variables, as different methods yield different extraction efficiencies.|All-grain brewing often produces more complex flavors, while extract brewing can result in cleaner, simpler flavors.|
|Pitch Rate|Amount of yeast pitched per gravity unit, often measured in millions of cells/ml/°P.|Influences fermentation rate, flavor development, and final gravity (FG), affecting the balance of the beer.|Higher pitch rates result in cleaner fermentations, while lower rates may enhance fruity or ester flavors.|
|Priming Method|Carbonation method used post-fermentation (e.g., sugar, forced carbonation).|Contributes to the carbonation level, which affects mouthfeel, aroma release, and sensory experience.|Natural priming creates finer bubbles, adding softness, while forced carbonation gives a sharper mouthfeel.|
|Priming Amount|Amount of priming sugar added for carbonation.|Helps achieve desired carbonation levels, enhancing texture and visual appearance.|More priming sugar results in higher carbonation, important for enhancing mouthfeel and releasing aromas.|

**Explanation of Secondary Variables’ Influence**
1. **Boil Size, Boil Time, and Boil Gravity** - These factors contribute directly to flavor concentration and bitterness extraction. They allow control over wort concentration, influencing the beer's balance between sweetness and bitterness, which is essential in hop-forward styles (e.g., IPAs) and darker, maltier styles (e.g., stouts).
2. **Mash Thickness** - Modifying mash thickness affects the extraction of sugars, which influences the body, sweetness, and dryness of the beer. For instance, a thicker mash retains more unfermentable sugars, resulting in a fuller-bodied beer, ideal for styles like porters and stouts.
3. **Sugar Scale** Ensures precision in measuring OG and FG, crucial for consistency in alcohol content and body. Accurate readings using a standardized scale (like Brix or Plato) enable reproducible recipes and controlled fermentation outcomes.
4. **Brew Method** - Influences the complexity of flavors and efficiency in extraction. All-grain brewing typically offers more depth and character, as grains are mashed and extracted fresh, while extract brewing provides simplicity, often yielding cleaner profiles.
5. **Pitch Rate** - Affects the speed and profile of fermentation. High pitch rates tend to produce cleaner, more neutral flavors by avoiding stress on the yeast, while lower rates can add complexity, increasing esters and phenols, particularly in styles like Belgian ales.
6. **Priming Method and Amount** - These control the beer's carbonation level, impacting mouthfeel, aroma, and visual appeal. Higher carbonation (from more priming sugar or natural carbonation) enhances aroma release and can add a creamy texture, especially desirable in styles like Belgian ales and wheat beers.

In [8]:
df_raw_style = pd.read_csv("../data/raw/styleData.csv", encoding='latin1')
df_raw_style.head()

,Style,StyleID
0,Altbier,1
1,Alternative Grain Beer,2
2,Alternative Sugar Beer,3
3,American Amber Ale,4
4,American Barleywine,5


In [9]:
df_raw_style.shape

(176, 2)

In [10]:
df_raw_style.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Style    175 non-null    object
 1   StyleID  176 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.9+ KB


In [12]:
df_raw.isnull().sum()

BeerID               0
Name                 2
URL                  0
Style              596
StyleID              0
Size(L)              0
OG                   0
FG                   0
ABV                  0
IBU                  0
Color                0
BoilSize             0
BoilTime             0
BoilGravity       2990
Efficiency           0
MashThickness    29864
SugarScale           0
BrewMethod           0
PitchRate        39252
PrimaryTemp      22662
PrimingMethod    67101
PrimingAmount    69087
UserId           50490
dtype: int64

In [26]:
df_raw.groupby('Style')[['BoilGravity', 'MashThickness']].apply(lambda x: x.isnull().sum())[:50]

,BoilGravity,MashThickness
Style,,
Altbier,0,42
Alternative Grain Beer,0,14
Alternative Sugar Beer,0,10
American Amber Ale,101,899
American Barleywine,22,127
American Brown Ale,72,504
American IPA,396,4841
American Lager,0,74
American Light Lager,0,826


In [27]:
df_raw.groupby('Style')[['BoilGravity', 'MashThickness']].apply(lambda x: x.isnull().sum())[50:100]

,BoilGravity,MashThickness
Style,,
Dark American Lager,7,24
Dark Mild,0,83
Doppelbock,9,92
Dortmunder Export,1,19
Double IPA,0,273
Dry Mead,6,55
Dry Stout,29,209
Dunkelweizen,25,106
Dunkles Bock,0,13


In [28]:
df_raw.groupby('Style')[['BoilGravity', 'MashThickness']].apply(lambda x: x.isnull().sum())[100:150]

,BoilGravity,MashThickness
Style,,
London Brown Ale,0,1
Maibock/Helles Bock,16,57
Metheglin,4,11
Mild,25,140
Mixed-Fermentation Sour Beer,0,54
Mixed-Style Beer,0,48
Munich Dunkel,6,87
Munich Helles,12,113
Märzen,0,107


In [29]:
df_raw.groupby('Style')[['BoilGravity', 'MashThickness']].apply(lambda x: x.isnull().sum())[150:]

,BoilGravity,MashThickness
Style,,
Specialty IPA: White IPA,0,81
Specialty Smoked Beer,0,27
Specialty Wood-Aged Beer,0,2
Spice Herb or Vegetable Beer,25,233
Standard American Lager,5,21
Standard/Ordinary Bitter,18,138
Straight (Unblended) Lambic,0,26
Strong Bitter,0,165
Strong Scotch Ale,26,92


In [16]:
df_raw['Style'].isnull().sum()

np.int64(596)